In [1]:

import pandas as pd
import numpy as np
from mealpy.optimizer import Optimizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import numpy as np
import time  # Import the time module

from mealpy.utils import io
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming you have a dataset 'X' and corresponding labels 'y'

# Assuming you have a dataset 'X' and corresponding labels 'y'

datasetOZ = pd.read_csv('labelled-balanced-data11.csv')
X, y = datasetOZ.iloc[:, :-1], datasetOZ.iloc[:, -1]

# Determine the number of features
num_features = X.shape[1]

In [2]:
class OriginalZOA(Optimizer):
    def __init__(self, epoch=10000, pop_size=100, **kwargs):
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [10, 10000])
        self.set_parameters(["epoch", "pop_size"])
        self.sort_flag = False
        
    def evolve(self, epoch):
        
        # PHASE1: Foraging Behaviour
        pop_new = []
        for idx in range(0, self.pop_size):
            r1 = np.round(1 + np.random.rand())
            pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.g_best[self.ID_POS] - r1 * self.pop[idx][self.ID_POS])   # Eq. 3
            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
            pop_new.append([pos_new, None])
            if self.mode not in self.AVAILABLE_MODES:
                target = self.get_target_wrapper(pos_new)
                self.pop[idx] = self.get_better_solution([pos_new, target], self.pop[idx])
        if self.mode in self.AVAILABLE_MODES:
            pop_new = self.update_target_wrapper_population(pop_new)
            self.pop = self.greedy_selection_population(self.pop, pop_new)
            
        # PHASE2: Defense strategies against predators
        kk = np.random.permutation(self.pop_size)[0]
        pop_new = []
        for idx in range(0, self.pop_size):
            if np.random.rand() < 0.5:
                # S1: the lion attacks the zebra and thus the zebra chooses an escape strategy
                r2 = 0.1
                pos_new = self.pop[idx][self.ID_POS] + r2 * (2 + np.random.rand(self.problem.n_dims) - 1) * (1 - (epoch+1)/self.epoch)*self.pop[idx][self.ID_POS]
            else:
                # S2: other predators attack the zebra and the zebra will choose the offensive strategy
                r2 = np.random.randint(1, 3)
                pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.pop[kk][self.ID_POS] - r2 * self.pop[idx][self.ID_POS])
            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
            pop_new.append([pos_new, None])
            if self.mode not in self.AVAILABLE_MODES:
                target = self.get_target_wrapper(pos_new)
                self.pop[idx] = self.get_better_solution([pos_new, target], self.pop[idx])
        if self.mode in self.AVAILABLE_MODES:
            pop_new = self.update_target_wrapper_population(pop_new)
            self.pop = self.greedy_selection_population(self.pop, pop_new)


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

# Assuming X and y are defined elsewhere in your code

# Define a fitness function that uses Recursive Feature Elimination (RFE) with Random Forest
def fitness_function(solution):
    selected_features = [i for i, selected in enumerate(solution) if selected]
    if len(selected_features) < 2:
        return 0.0  # No features selected, return fitness value of 0
    
    X_subset = X.iloc[:, selected_features]  # Select the corresponding columns from the dataset

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.25, random_state=42)

    # Create a Random Forest classifier
    rfc = RandomForestClassifier()

    # Use Recursive Feature Elimination (RFE)
    rfe = RFE(estimator=rfc, n_features_to_select=len(selected_features))
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    X_test_rfe = rfe.transform(X_test)

    # Train the classifier on the subset of features selected by RFE
    rfc.fit(X_train_rfe, y_train)

    # Predict the labels for the test set
    y_pred = rfc.predict(X_test_rfe)

    # Calculate the accuracy score as the fitness value
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Assuming num_features is defined elsewhere in your code
num_features = X.shape[1]

problemROZ = {
    "fit_func": fitness_function,
    "lb": [0] * num_features,  # Lower bound for each feature (0 means not selected)
    "ub": [1] * num_features,  # Upper bound for each feature (1 means selected)
    "minmax": "max",  # Maximizing the fitness function
    "save_population": True
}

# Number of optimization runs
num_runs = 10

# Store results in a list of dictionaries
all_results_rf_zoa = []

for run in range(num_runs):
    # Create the OriginalZOA optimization model
    modelROZ = OriginalZOA(epoch=100, pop_size=100)

    # Measure the total execution time
    start_time = time.time()

    # Solve the optimization problem
    best_position, best_fitness = modelROZ.solve(problemROZ)

    # Calculate and print the total execution time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Run {run + 1}: Total execution time: {total_time} seconds")

    # Create a dictionary to map feature names to their corresponding position values
    feature_position_dict = {feature_name: position for feature_name, position in zip(X.columns, best_position)}

    # Store results
    all_results_rf_zoa.append({"position_values": feature_position_dict, "fitness": best_fitness})

    


2024/01/15 02:16:02 PM, INFO, __main__.OriginalZOA: Solving single objective optimization problem.
2024/01/15 02:23:25 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 1, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 286.23904 seconds
2024/01/15 02:27:57 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 2, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 271.61833 seconds
2024/01/15 02:32:25 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 3, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 267.91230 seconds
2024/01/15 02:36:56 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 4, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 271.28647 seconds
2024/01/15 02:41:25 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 5, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 268.46733 seconds
2024/01/15 02:46:10 PM, INFO, __main__.OriginalZOA: >Pro

2024/01/15 05:36:11 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 49, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 238.49260 seconds
2024/01/15 05:40:10 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 50, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 239.06833 seconds
2024/01/15 05:44:07 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 51, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 237.03743 seconds
2024/01/15 05:48:09 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 52, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 241.92365 seconds
2024/01/15 05:52:13 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 53, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 244.21226 seconds
2024/01/15 05:56:14 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 54, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime

2024/01/15 08:55:35 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 98, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 242.94194 seconds
2024/01/15 08:59:40 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 99, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 245.27376 seconds
2024/01/15 09:03:42 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 100, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 241.84509 seconds


Run 1: Total execution time: 24462.074977636337 seconds


2024/01/15 09:03:44 PM, INFO, __main__.OriginalZOA: Solving single objective optimization problem.
2024/01/15 09:10:07 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 1, Current best: 0.860655737704918, Global best: 0.860655737704918, Runtime: 261.84858 seconds
2024/01/15 09:14:15 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 2, Current best: 0.8611680327868853, Global best: 0.8611680327868853, Runtime: 247.23712 seconds
2024/01/15 09:18:07 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 3, Current best: 0.8611680327868853, Global best: 0.8611680327868853, Runtime: 232.42611 seconds
2024/01/15 09:22:01 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 4, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 234.26079 seconds
2024/01/15 09:26:02 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 5, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 240.96394 seconds
2024/01/15 09:30:00 PM, INFO, __main__.OriginalZOA: >Probl

...

Run 9: Total execution time: 24379.983429670334 seconds


2024/01/18 03:10:00 AM, INFO, __main__.OriginalZOA: Solving single objective optimization problem.
2024/01/18 03:15:52 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 1, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 228.49535 seconds
2024/01/18 03:19:50 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 2, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 237.78134 seconds
2024/01/18 03:23:47 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 3, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 236.96222 seconds
2024/01/18 03:27:47 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 4, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 240.30117 seconds
2024/01/18 03:31:43 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 5, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 236.58002 seconds
2024/01/18 03:35:41 AM, INFO, __main__.OriginalZOA: >Pro

2024/01/18 06:25:03 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 49, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 233.75980 seconds
2024/01/18 06:29:02 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 50, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 239.63600 seconds
2024/01/18 06:32:58 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 51, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 236.22186 seconds
2024/01/18 06:36:56 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 52, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 237.38107 seconds
2024/01/18 06:40:57 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 53, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 241.15876 seconds
2024/01/18 06:44:58 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 54, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime

2024/01/18 09:41:09 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 98, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 238.77555 seconds
2024/01/18 09:45:05 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 99, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 236.31820 seconds
2024/01/18 09:49:09 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 100, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 243.36189 seconds


Run 10: Total execution time: 23950.117335796356 seconds


In [4]:
# Print the feature names and corresponding position values for each run in sorted order
for run, result in enumerate(all_results_rf_zoa, start=1):
    sorted_features = sorted(result["position_values"].items(), key=lambda x: x[1])
    print(f"\nRun {run}: Fitness (Accuracy): {result['fitness']}")
    for feature_name, position_value in sorted_features:
        print(f"Feature: {feature_name}, Position Value: {position_value}")


Run 1: Fitness (Accuracy): 0.8642418032786885
Feature: Gender, Position Value: 0.1689089938074158
Feature: BP, Position Value: 0.2365863046251889
Feature: BMI, Position Value: 0.2472122688714915
Feature: Region, Position Value: 0.3962883818061935
Feature: Physical_Activity, Position Value: 0.44346091092246037
Feature: Age, Position Value: 0.47151318735397796
Feature: Waist_Size, Position Value: 0.4899285779407738
Feature: Diet, Position Value: 0.5154571086277961
Feature: Family_History, Position Value: 0.6889542184547939
Feature: Smoking, Position Value: 0.9294426653192142

Run 2: Fitness (Accuracy): 0.8637295081967213
Feature: Family_History, Position Value: 0.2589675047124649
Feature: Gender, Position Value: 0.3023623666857881
Feature: Physical_Activity, Position Value: 0.3805733089271514
Feature: Age, Position Value: 0.38420764746696257
Feature: BMI, Position Value: 0.4133520471651652
Feature: Diet, Position Value: 0.6032830818524987
Feature: Region, Position Value: 0.6145697825494

In [7]:
# Create a DataFrame from the results
results_df = pd.DataFrame(all_results_rf_zoa)

# Save the DataFrame to a CSV file
results_df.to_csv('optimization_results RFE ZOA last final.csv', index=False)

# Print the feature names and corresponding position values for each run
for run, result in enumerate(all_results_rf_zoa, start=1):
    print(f"\nRun {run}: Fitness (Accuracy): {result['fitness']}")
    for feature_name, position_value in result['position_values'].items():
        print(f"Feature: {feature_name}, Position Value: {position_value}")



Run 1: Fitness (Accuracy): 0.8642418032786885
Feature: Region, Position Value: 0.3962883818061935
Feature: Age, Position Value: 0.47151318735397796
Feature: Gender, Position Value: 0.1689089938074158
Feature: BMI, Position Value: 0.2472122688714915
Feature: Waist_Size, Position Value: 0.4899285779407738
Feature: Physical_Activity, Position Value: 0.44346091092246037
Feature: Diet, Position Value: 0.5154571086277961
Feature: BP, Position Value: 0.2365863046251889
Feature: Family_History, Position Value: 0.6889542184547939
Feature: Smoking, Position Value: 0.9294426653192142

Run 2: Fitness (Accuracy): 0.8637295081967213
Feature: Region, Position Value: 0.614569782549491
Feature: Age, Position Value: 0.38420764746696257
Feature: Gender, Position Value: 0.3023623666857881
Feature: BMI, Position Value: 0.4133520471651652
Feature: Waist_Size, Position Value: 1.0
Feature: Physical_Activity, Position Value: 0.3805733089271514
Feature: Diet, Position Value: 0.6032830818524987
Feature: BP, Pos